In [ ]:
"""
Stage 1: Train the geometry MLP.

You can monitor the training phase by the command "tensorboard --logdir runs"

An epoch takes around 1 hour for 400 x 400 images using a single NVIDIA RTX 2070 GPU with 8GB of RAM.

You can increase the batch size if you have access to more memory to speed up training.
"""

!python pipeline.py --stage geometry --prefix test --suffix density --dataset_name blender --root_dir datasets/nerf_synthetic/lego --img_wh 400 400 --num_epochs_density 1 --batch_size 1024 --lr 5e-4

In [ ]:
"""
Stage 2: Train the style MLP. 

Three options for the flag --style_mode:
    - small: Work with 50 x 50 images, takes < 5 mins. Recommended lr: 1e-3
    - patch: Work with 50 x 50 patches, takes around 30 mins. Recommended lr: 1e-4
    - memory_saving: Work with complete images, takes around 50 mins. Recommended lr: 1e-3
"""

!python pipeline.py --stage style --prefix test --suffix style --style_dir starry_night.jpg --style_mode memory_saving --coarse_path ckpts/test_nerf_coarse_density.pt --fine_path ckpts/test_nerf_fine_density.pt --dataset_name blender --root_dir datasets/nerf_synthetic/lego --img_wh 400 400 --num_epochs_style 1 --lr 1e-3

In [ ]:
# create a gif out of the rendered images from camere poses in test dataset

!python eval.py --dataset_name blender --root_dir datasets/nerf_synthetic/lego --scene_name lego --img_wh 400 400 --coarse_path ckpts/test_nerf_coarse_style.pt --fine_path ckpts/test_nerf_fine_style.pt --gif_name gif

In [6]:
# display the saved gif

from IPython.core.display import display, HTML
display(HTML('<h1><img src="gif.gif" width="400" align="center"></h1>'))